# Spike Presence/Absence Model
This notebook uses the Gravity mmodel and Mote Carlo Simulation to predict the probability of spikes in concentrations of PM2.5 from PurpleAir sensor data in Minneapolis

In [2]:
# Libraries
import pandas as pd
import numpy as np
import arcpy
import os

In [3]:
# Set working Directory

# Get CWD

cwd = os.getcwd() # This is a global variable for where the notebook is (must change if running in arcpro)

# Create GeoDataBase
# This is the communal GeoDataBase, only run once

if not os.path.exists(os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')): # If it doesn't exist, create it

    arcpy.management.CreateFileGDB(os.path.join(cwd, '..', '..', 'data'), 'QAQC')

# Make it workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

arcpy.env.overwriteOutput = True # Overwrite layers is okay

In [5]:
cwd

'c:\\Users\\MrJDF\\Desktop\\QualityAirQualityCities\\arcpy\\Modeling'

## Load Data

In [6]:
# set variable to navigate to data folder
data_path = os.path.join(cwd,'..', '..', 'data')

In [10]:

# grab the data frame

'''
This should be a df of the previous 24 hr values. 
Need to create a workflow/notebok that does this in real time. 
Right now we're using the data Ethan cleaned for the april 01 2023 data.
'''

spike_wind_data = pd.read_csv(os.path.join(data_path, 'station_spike_wind_data.csv'))

In [8]:
spike_wind_data.head()

,sensor_index,date,is_weekday,n_observations,pm25_fullDay_mean,pm25_fullDay_minutesAbove12ug,n_spikes,humidity_fullDay_mean,temperature_fullDay_mean,pressure_fullDay_mean,idwSum_facilities,idwSum_traffic,daily_wind_speed_avg,daily_wind_vector_direction_avg,latitude,longitude
0,143656,3/18/2023,0,144,5.341125,190,0,45.833,23.505,988.205,0.005054,861.930809,9.625000,197.291667,44.932407,-93.28342
1,143656,11/5/2022,0,144,8.760681,290,0,60.125,44.010,971.738,0.005054,861.930809,9.712500,262.250000,44.932407,-93.28342
2,143656,12/6/2022,1,144,9.107910,340,0,49.004,27.014,986.269,0.005054,861.930809,8.933333,271.375000,44.932407,-93.28342
3,143656,12/7/2022,1,144,14.958146,610,14,49.725,28.465,993.579,0.005054,861.930809,9.012500,270.208333,44.932407,-93.28342
4,143656,1/22/2023,0,144,26.534174,1430,55,61.631,28.517,984.403,0.005054,861.930809,9.445833,291.958333,44.932407,-93.28342


In [22]:
# unique_stations = spike_wind_data['sensor_index'].unique()
# mask = spike_wind_data['sensor_index'].isin(unique_stations)
unique_stations = spike_wind_data.drop_duplicates(subset='sensor_index' , keep = 'first')

In [23]:
len(unique_stations)

53

In [33]:

from scipy.spatial.distance import cdist
from pprint import pprint 

locations = unique_stations[['latitude', 'longitude']].values
distance_matrix = cdist(locations, locations, metric='euclidean')
pprint(distance_matrix)

array([[0.        , 0.06113865, 0.0572818 , ..., 0.03984551, 0.03965668,
        0.04088336],
       [0.06113865, 0.        , 0.10698803, ..., 0.07594077, 0.1004073 ,
        0.06066351],
       [0.0572818 , 0.10698803, 0.        , ..., 0.03293513, 0.03740579,
        0.05070925],
       ...,
       [0.03984551, 0.07594077, 0.03293513, ..., 0.        , 0.04812616,
        0.01780165],
       [0.03965668, 0.1004073 , 0.03740579, ..., 0.04812616, 0.        ,
        0.06167653],
       [0.04088336, 0.06066351, 0.05070925, ..., 0.01780165, 0.06167653,
        0.        ]])
